# תרגיל למידת מכונה - פתרון מלא

## תוכן העניינים
- **חלק א':** ניבוי מחירי דיור - רגרסיה ליניארית
- **חלק ב':** ניבוי התקפי לב - רגרסיה לוגיסטית

---
## הכנות סביבה וייבואים

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_curve, auc
)

# הגדרת סגנון כללי לגרפים
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 11

---
# חלק א': ניבוי מחירי דיור - רגרסיה ליניארית

### מה זה רגרסיה ליניארית?
מודל שמנסה למצוא קשר ליניארי (קו ישר) בין משתנים מסבירים (features) למשתנה תלוי (target).

**הנוסחה:** `price = b0 + b1*bedrooms + b2*bathrooms + b3*sqft_living + ...`

המודל ממזער את **סכום ריבועי השגיאות** (OLS - Ordinary Least Squares).

### שלב 1: טעינת הנתונים

In [ ]:
df_housing = pd.read_csv("Project housing data .csv")

print(f"מספר שורות: {df_housing.shape[0]}, מספר עמודות: {df_housing.shape[1]}")
print(f"\nעמודות: {list(df_housing.columns)}")
df_housing.head()

In [ ]:
df_housing.describe()

### שלב 2: עיבוד מקדים

In [ ]:
# בדיקת ערכים חסרים
print("ערכים חסרים בכל עמודה:")
print(df_housing.isnull().sum())

# הסבר על קידוד משתנים:
# - waterfront: משתנה בינארי (0/1) - כבר מקודד
# - view (0-4): משתנה סדור (ordinal)
# - condition (1-5): משתנה סדור (ordinal)
# - שאר המשתנים: מספריים רציפים

### שלב 3: הגדרת משתנים והפרדת נתונים

**הסבר:** מחלקים את הנתונים ל-80% אימון / 20% מבחן. המודל לומד רק מסט האימון, ונבדק אותו על סט המבחן (שלא נראה באימון).

In [ ]:
# X = כל המשתנים המסבירים, y = המשתנה התלוי (מחיר)
X_housing = df_housing.drop("price", axis=1)
y_housing = df_housing["price"]

# חלוקה לסט אימון (80%) וסט מבחן (20%)
# random_state=42 מבטיח תוצאות עקביות בכל הרצה
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(
    X_housing, y_housing, test_size=0.2, random_state=42
)

print(f"גודל סט אימון: {X_train_h.shape[0]} דירות")
print(f"גודל סט מבחן: {X_test_h.shape[0]} דירות")

### שלב 4: בניית מודל רגרסיה ליניארית

**הסבר:** רגרסיה ליניארית מנסה למצוא קו (או מישור) שממזער את סכום ריבועי השגיאות.

In [ ]:
model_housing = LinearRegression()
model_housing.fit(X_train_h, y_train_h)

# הצגת המקדמים (coefficients)
print("מקדמי המודל:")
for feature, coef in zip(X_housing.columns, model_housing.coef_):
    print(f"  {feature}: {coef:,.2f}")
print(f"  חותך (intercept): {model_housing.intercept_:,.2f}")

### שלב 5: הערכת ביצועי המודל

| מדד | משמעות |
|------|----------|
| **MAE** | שגיאה מוחלטת ממוצעת - "בממוצע, המודל טועה ב-X דולר" |
| **MSE** | שגיאה ריבועית ממוצעת - מענישה טעויות גדולות יותר |
| **RMSE** | שורש MSE - ביחידות של דולר |
| **R²** | מקדם הקביעה - "המודל מסביר X% מהשונות" |

In [ ]:
# החיזוי נעשה על נתוני המבחן (שהמודל לא ראה בזמן האימון)
y_pred_h = model_housing.predict(X_test_h)

mae = mean_absolute_error(y_test_h, y_pred_h)
mse = mean_squared_error(y_test_h, y_pred_h)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_h, y_pred_h)

print("הערכת ביצועי המודל (על נתוני המבחן):")
print(f"  MAE (שגיאה מוחלטת ממוצעת): {mae:,.2f} $")
print(f"    -> בממוצע, המודל טועה ב-{mae:,.0f} דולר")
print(f"  MSE: {mse:,.2f}")
print(f"  RMSE: {rmse:,.2f} $")
print(f"    -> מדד שגיאה בדולר, מעניש טעויות גדולות יותר")
print(f"  R\u00b2 (מקדם הקביעה): {r2:.4f}")
print(f"    -> המודל מסביר {r2*100:.1f}% מהשונות במחירי הדירות")

### שלב 6: פרדיקציה לדירה חדשה

In [ ]:
# דירה חדשה עם מאפיינים ריאליסטיים
new_apartment = pd.DataFrame({
    "bedrooms": [3], "bathrooms": [2], "sqft_living": [1800],
    "sqft_lot": [6000], "floors": [1], "waterfront": [0],
    "view": [2], "condition": [4], "sqft_above": [1800],
    "sqft_basement": [0], "yr_built": [1990]
})

predicted_price = model_housing.predict(new_apartment)[0]

print("מאפייני הדירה החדשה:")
print(f"  חדרי שינה: 3, חדרי אמבטיה: 2")
print(f"  שטח מגורים: 1,800 רגל רבוע")
print(f"  נוף: 2 (בינוני), מצב: 4 (טוב), שנת בנייה: 1990")
print(f"\n  מחיר חזוי: ${predicted_price:,.2f}")

avg_price = df_housing["price"].mean()
median_price = df_housing["price"].median()
print(f"\n  לשם השוואה:")
print(f"    מחיר ממוצע: ${avg_price:,.2f}")
print(f"    מחיר חציוני: ${median_price:,.2f}")

### ויזואליזציה - דשבורד מחירי דיור

In [ ]:
fig_housing, axes_h = plt.subplots(2, 3, figsize=(20, 12))
fig_housing.suptitle("Dashboard - Housing Price Analysis", fontsize=18, fontweight="bold", y=1.02)

# גרף 1: התפלגות מחירים
avg_price = df_housing["price"].mean()
median_price = df_housing["price"].median()
axes_h[0, 0].hist(df_housing["price"], bins=40, color="steelblue", edgecolor="white", alpha=0.8)
axes_h[0, 0].axvline(avg_price, color="red", linestyle="--", linewidth=2, label=f"Mean: ${avg_price:,.0f}")
axes_h[0, 0].axvline(median_price, color="orange", linestyle="--", linewidth=2, label=f"Median: ${median_price:,.0f}")
axes_h[0, 0].set_title("Price Distribution", fontsize=13, fontweight="bold")
axes_h[0, 0].set_xlabel("Price ($)")
axes_h[0, 0].set_ylabel("Count")
axes_h[0, 0].legend()

# גרף 2: מטריצת קורלציה
corr_matrix = df_housing.corr()
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="RdBu_r", center=0,
            ax=axes_h[0, 1], cbar_kws={"shrink": 0.8}, linewidths=0.5, square=True,
            annot_kws={"size": 7})
axes_h[0, 1].set_title("Correlation Matrix", fontsize=13, fontweight="bold")
axes_h[0, 1].tick_params(axis="both", labelsize=7)

# גרף 3: מחיר בפועל מול מחיר חזוי
axes_h[0, 2].scatter(y_test_h, y_pred_h, alpha=0.5, color="steelblue", edgecolor="white", s=30)
max_val = max(y_test_h.max(), y_pred_h.max())
axes_h[0, 2].plot([0, max_val], [0, max_val], "r--", linewidth=2, label="Perfect Prediction")
axes_h[0, 2].set_title(f"Actual vs Predicted (R\u00b2={r2:.3f})", fontsize=13, fontweight="bold")
axes_h[0, 2].set_xlabel("Actual Price ($)")
axes_h[0, 2].set_ylabel("Predicted Price ($)")
axes_h[0, 2].legend()

# גרף 4: מקדמי המודל
coef_df = pd.DataFrame({"Feature": X_housing.columns, "Coefficient": model_housing.coef_}).sort_values("Coefficient", key=abs, ascending=True)
colors = ["#e74c3c" if c < 0 else "#2ecc71" for c in coef_df["Coefficient"]]
axes_h[1, 0].barh(coef_df["Feature"], coef_df["Coefficient"], color=colors, edgecolor="white")
axes_h[1, 0].set_title("Model Coefficients", fontsize=13, fontweight="bold")
axes_h[1, 0].set_xlabel("Coefficient Value")
axes_h[1, 0].axvline(0, color="black", linewidth=0.8)

# גרף 5: התפלגות השגיאות
residuals = y_test_h - y_pred_h
axes_h[1, 1].hist(residuals, bins=40, color="coral", edgecolor="white", alpha=0.8)
axes_h[1, 1].axvline(0, color="black", linestyle="--", linewidth=2)
axes_h[1, 1].set_title("Residuals Distribution", fontsize=13, fontweight="bold")
axes_h[1, 1].set_xlabel("Prediction Error ($)")
axes_h[1, 1].set_ylabel("Count")

# גרף 6: שטח מגורים מול מחיר
axes_h[1, 2].scatter(df_housing["sqft_living"], df_housing["price"],
                     alpha=0.4, color="steelblue", edgecolor="white", s=20)
axes_h[1, 2].set_title("Sqft Living vs Price", fontsize=13, fontweight="bold")
axes_h[1, 2].set_xlabel("Sqft Living")
axes_h[1, 2].set_ylabel("Price ($)")

plt.tight_layout()
plt.savefig("housing_dashboard.png", dpi=150, bbox_inches="tight")
plt.show()

---
# חלק ב': ניבוי התקפי לב - רגרסיה לוגיסטית

### מה זה רגרסיה לוגיסטית?
בניגוד לרגרסיה ליניארית (שמנבאת ערך רציף), רגרסיה לוגיסטית מחשבת **הסתברות** לשייכות לקטגוריה (0 או 1).

**הנוסחה:** מחשבת `P(y=1)` באמצעות פונקציית sigmoid.

### שלב 1: טעינת הנתונים

In [ ]:
df_heart = pd.read_csv("Project Heart attack data.csv")

print(f"מספר שורות: {df_heart.shape[0]}, מספר עמודות: {df_heart.shape[1]}")
print(f"\nהתפלגות משתנה המטרה (target):")
print(df_heart["target"].value_counts())
print("  0 = סיכוי נמוך להתקף לב")
print("  1 = סיכוי גבוה להתקף לב")
df_heart.head()

### שלב 2: קידוד משתנים קטגוריאליים (One-Hot Encoding)

**הסבר:**
- `cp` (סוג כאב חזה) - משתנה נומינלי (0,1,2,3) - אין סדר טבעי, לכן One-Hot
- `restecg` (תוצאות ECG) - משתנה נומינלי (0,1,2) - One-Hot
- `sex`, `fbs`, `exng` - בינאריים (0/1) - כבר מקודדים

`drop_first=True` מונע בעיית מולטיקוליניאריות.

In [ ]:
# בדיקת ערכים חסרים
print("ערכים חסרים:")
print(df_heart.isnull().sum())

# One-Hot Encoding
df_heart_encoded = pd.get_dummies(df_heart, columns=["cp", "restecg"], drop_first=True)

print(f"\nלאחר קידוד One-Hot:")
print(f"  עמודות: {df_heart_encoded.shape[1]} (לפני: {df_heart.shape[1]})")

### שלב 3: הגדרת משתנים וחלוקה

In [ ]:
X_heart = df_heart_encoded.drop("target", axis=1)
y_heart = df_heart_encoded["target"]

X_train_hr, X_test_hr, y_train_hr, y_test_hr = train_test_split(
    X_heart, y_heart, test_size=0.2, random_state=42
)

print(f"גודל סט אימון: {X_train_hr.shape[0]} מטופלים")
print(f"גודל סט מבחן: {X_test_hr.shape[0]} מטופלים")

### שלב 4: בניית מודל רגרסיה לוגיסטית

In [ ]:
# max_iter=1000 כדי להבטיח שהאלגוריתם יתכנס
model_heart = LogisticRegression(max_iter=1000, random_state=42)
model_heart.fit(X_train_hr, y_train_hr)
print("מודל רגרסיה לוגיסטית אומן בהצלחה!")

### שלב 5: הערכת ביצועים

| מדד | משמעות |
|------|----------|
| **Accuracy** | אחוז סיווגים נכונים מתוך הכל |
| **Precision** | מתוך מי שהמודל חזה שבסיכון, כמה באמת בסיכון |
| **Recall** | מתוך כל החולים האמיתיים, כמה המודל זיהה |
| **F1** | ממוצע הרמוני של Precision ו-Recall |

In [ ]:
y_pred_hr = model_heart.predict(X_test_hr)

accuracy = accuracy_score(y_test_hr, y_pred_hr)
precision = precision_score(y_test_hr, y_pred_hr)
recall = recall_score(y_test_hr, y_pred_hr)
f1 = f1_score(y_test_hr, y_pred_hr)

print("הערכת ביצועים (על נתוני המבחן):")
print(f"  Accuracy: {accuracy:.4f} -> {accuracy*100:.1f}% מהתצפיות סווגו נכון")
print(f"  Precision: {precision:.4f} -> מתוך מי שהמודל חזה שבסיכון, {precision*100:.1f}% באמת")
print(f"  Recall: {recall:.4f} -> המודל זיהה {recall*100:.1f}% מכלל החולים")
print(f"  F1 Score: {f1:.4f}")

cm = confusion_matrix(y_test_hr, y_pred_hr)
print(f"\nמטריצת בלבול:")
print(f"  [TN={cm[0,0]}, FP={cm[0,1]}]")
print(f"  [FN={cm[1,0]}, TP={cm[1,1]}]")

print(f"\nדוח סיווג מלא:")
print(classification_report(y_test_hr, y_pred_hr, target_names=["סיכוי נמוך", "סיכוי גבוה"]))

### איזו מטריקה חשובה יותר בהקשר הרפואי?

**Recall (רגישות)** היא המטריקה החשובה ביותר:

- **False Negative** (השגיאה החמורה ביותר) = מטופל בסיכון אבל המודל אמר שהוא בסדר → **עלול לעלות בחיים!**
- **False Positive** = המודל חזה סיכון אבל המטופל בריא → בדיקות מיותרות, פחות חמור

**Recall מודד: מתוך כל החולים האמיתיים, כמה המודל הצליח לזהות?**

### שלב 6: פרדיקציה למטופל חדש

In [ ]:
# מטופל חדש: גבר בן 55, כאב חזה אסימפטומטי, לחץ דם 140, כולסטרול 250
new_patient_data = {
    "age": [55], "sex": [1], "trtbps": [140], "chol": [250],
    "fbs": [0], "thalachh": [150], "exng": [0], "ca": [1],
}

# הוספת עמודות One-Hot
for col in X_heart.columns:
    if col not in new_patient_data:
        new_patient_data[col] = [0]

new_patient = pd.DataFrame(new_patient_data)[X_heart.columns]

prediction = model_heart.predict(new_patient)[0]
prediction_proba = model_heart.predict_proba(new_patient)[0]

print("מאפייני המטופל:")
print(f"  גיל: 55, מין: גבר, לחץ דם: 140, כולסטרול: 250")
print(f"  דופק מקסימלי: 150, כלי דם ראשיים: 1")
print(f"\n  תוצאת החיזוי: {'סיכוי גבוה' if prediction == 1 else 'סיכוי נמוך'}")
print(f"  הסתברות לסיכון נמוך: {prediction_proba[0]*100:.1f}%")
print(f"  הסתברות לסיכון גבוה: {prediction_proba[1]*100:.1f}%")

### ויזואליזציה - דשבורד התקפי לב

In [ ]:
fig_heart, axes_hr = plt.subplots(2, 3, figsize=(20, 12))
fig_heart.suptitle("Dashboard - Heart Attack Prediction Analysis", fontsize=18, fontweight="bold", y=1.02)

# גרף 1: מטריצת בלבול
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=axes_hr[0, 0],
            xticklabels=["Low Risk", "High Risk"],
            yticklabels=["Low Risk", "High Risk"],
            annot_kws={"size": 16}, linewidths=2)
axes_hr[0, 0].set_title("Confusion Matrix", fontsize=13, fontweight="bold")
axes_hr[0, 0].set_xlabel("Predicted", fontsize=11)
axes_hr[0, 0].set_ylabel("Actual", fontsize=11)

# גרף 2: מדדי ביצועים
metrics_names = ["Accuracy", "Precision", "Recall", "F1 Score"]
metrics_values = [accuracy, precision, recall, f1]
bar_colors = ["#3498db", "#2ecc71", "#e74c3c", "#9b59b6"]
bars = axes_hr[0, 1].bar(metrics_names, metrics_values, color=bar_colors, edgecolor="white", width=0.6)
for bar, val in zip(bars, metrics_values):
    axes_hr[0, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                       f"{val:.3f}", ha="center", fontweight="bold", fontsize=11)
axes_hr[0, 1].set_ylim(0, 1.1)
axes_hr[0, 1].set_title("Model Performance Metrics", fontsize=13, fontweight="bold")
axes_hr[0, 1].set_ylabel("Score")

# גרף 3: עקומת ROC
y_proba_hr = model_heart.predict_proba(X_test_hr)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test_hr, y_proba_hr)
roc_auc = auc(fpr, tpr)
axes_hr[0, 2].plot(fpr, tpr, color="#e74c3c", linewidth=2.5, label=f"ROC Curve (AUC = {roc_auc:.3f})")
axes_hr[0, 2].plot([0, 1], [0, 1], "k--", linewidth=1, alpha=0.5, label="Random Classifier")
axes_hr[0, 2].fill_between(fpr, tpr, alpha=0.15, color="#e74c3c")
axes_hr[0, 2].set_title("ROC Curve", fontsize=13, fontweight="bold")
axes_hr[0, 2].set_xlabel("False Positive Rate")
axes_hr[0, 2].set_ylabel("True Positive Rate")
axes_hr[0, 2].legend(loc="lower right")

# גרף 4: התפלגות גילאים לפי סיכון
for target_val, label, color in [(0, "Low Risk", "#2ecc71"), (1, "High Risk", "#e74c3c")]:
    subset = df_heart[df_heart["target"] == target_val]
    axes_hr[1, 0].hist(subset["age"], bins=15, alpha=0.6, color=color, label=label, edgecolor="white")
axes_hr[1, 0].set_title("Age Distribution by Risk", fontsize=13, fontweight="bold")
axes_hr[1, 0].set_xlabel("Age")
axes_hr[1, 0].set_ylabel("Count")
axes_hr[1, 0].legend()

# גרף 5: מקדמי המודל
heart_coef_df = pd.DataFrame({"Feature": X_heart.columns, "Coefficient": model_heart.coef_[0]}).sort_values("Coefficient", key=abs, ascending=True)
colors_hr = ["#e74c3c" if c < 0 else "#2ecc71" for c in heart_coef_df["Coefficient"]]
axes_hr[1, 1].barh(heart_coef_df["Feature"], heart_coef_df["Coefficient"], color=colors_hr, edgecolor="white")
axes_hr[1, 1].set_title("Model Coefficients", fontsize=13, fontweight="bold")
axes_hr[1, 1].set_xlabel("Coefficient Value")
axes_hr[1, 1].axvline(0, color="black", linewidth=0.8)

# גרף 6: התפלגות משתנה המטרה
target_counts = df_heart["target"].value_counts()
axes_hr[1, 2].pie(target_counts, labels=["High Risk (1)", "Low Risk (0)"],
                  colors=["#e74c3c", "#2ecc71"], autopct="%1.1f%%",
                  startangle=90, textprops={"fontsize": 12},
                  explode=(0.05, 0), shadow=True)
axes_hr[1, 2].set_title("Target Distribution", fontsize=13, fontweight="bold")

plt.tight_layout()
plt.savefig("heart_attack_dashboard.png", dpi=150, bbox_inches="tight")
plt.show()